In [91]:
import gzip
import numpy as np
from collections import defaultdict

def readGz(f):
  for l in gzip.open(f):
    yield eval(l)

In [92]:
u_i_r = []
u_bought_rating = defaultdict(dict)
i_be_bought_rating = defaultdict(dict)

sum_r_ui = 0
for l in readGz("train.json.gz"):
    u_i_r.append([l['reviewerID'],l['itemID'],l['rating']])
    u_bought_rating[l['reviewerID']][l['itemID']]= l['rating']
    i_be_bought_rating[l['itemID']][l['reviewerID']] = l['rating']
    sum_r_ui += l['rating']

In [93]:
train = u_i_r
validation = u_i_r[len(u_i_r)//2:]

In [94]:
k = 1
alpha_global = sum_r_ui/200000
alpha = alpha_global
beta_u = defaultdict(int)
beta_i = defaultdict(int)
gamma_u = defaultdict(lambda:np.random.uniform(-1, 1)) 
gamma_i = defaultdict(lambda:np.random.uniform(-1, 1)) 

In [95]:
print(gamma_u['s'])
print(gamma_i['t'])
np.dot(gamma_u['s'],gamma_i['t'])

0.16866604152526143
0.08705695561601456


0.014683552090993554

In [96]:
def mse():
    error = 0
    for u,i,r in validation:
        predict = alpha + beta_u[u] + beta_i[i] + gamma_u[u]*gamma_i[i]
        error += (r - predict)**2
    mse = error/len(validation)
    print("mse: {}".format(mse))

In [97]:
def als(lamda, fix_u,alpha,beta_u,beta_i,gamma_u,gamma_i):
    if not fix_u:
        for u in u_bought_rating:
            sum_gm = 0
            sum_bias = 0
            for i in u_bought_rating[u]:
                sum_gm += gamma_i[i]*gamma_i[i]
                sum_bias += gamma_i[i]*(u_bought_rating[u][i]-alpha-beta_u[u]-beta_i[i])
            gamma_u[u] = sum_bias/(lamda+sum_gm)
   
    else :
        for i in i_be_bought_rating:
            sum_gm = 0
            sum_bias = 0
            for u in i_be_bought_rating[i]:
                sum_gm += gamma_u[u]*gamma_u[u]
                sum_bias += gamma_u[u]*(i_be_bought_rating[i][u]-alpha-beta_u[u]-beta_i[i])
            gamma_i[i] = sum_bias/(lamda + sum_gm)
    # update alpha
    bias = 0
    for u,i,r in train:
        bias += (r-beta_u[u]-beta_i[i]-gamma_u[u]*gamma_i[i])
    alpha = bias/len(train)
    
    # update beta u 
    for u in u_bought_rating :
        bias = 0
        for i in u_bought_rating[u]:
            bias += (u_bought_rating[u][i]-alpha-beta_i[i]-gamma_u[u]*gamma_i[i])
        beta_u[u] = bias/(lamda+len(u_bought_rating[u]))
    
    # update beta i
    for i in i_be_bought_rating:
        bias =0
        for u in i_be_bought_rating[i]:
            bias += (i_be_bought_rating[i][u]-alpha -beta_u[u]-gamma_u[u]*gamma_i[i])
        beta_i[i] = bias/(lamda +len(i_be_bought_rating[i]))
    return alpha,beta_u,beta_i,gamma_u,gamma_i

In [104]:
k = 1
alpha = alpha_global
beta_u = defaultdict(int)
beta_i = defaultdict(int)
gamma_u = defaultdict(lambda:np.random.uniform(-0.3, 0.3)) 
gamma_i = defaultdict(lambda:np.random.uniform(-0.3, 0.3)) 
fix_u = False
for i in range(100):
    fix_u != fix_u
    alpha,beta_u,beta_i,gamma_u,gamma_i=als(6.4,fix_u,alpha,beta_u,beta_i,gamma_u,gamma_i)
    mse()

mse: 0.8159691869613601
mse: 0.815223743567878
mse: 0.8152055252756053
mse: 0.815187460425601
mse: 0.8151809043469614
mse: 0.8151788960153965
mse: 0.8151783253169359
mse: 0.8151781888673203
mse: 0.815178179805331
mse: 0.8151782036639175
mse: 0.8151782314213538
mse: 0.8151782549308831
mse: 0.8151782728048734
mse: 0.8151782857106168
mse: 0.8151782947638367
mse: 0.8151783010035321
mse: 0.8151783052555182
mse: 0.8151783081312606
mse: 0.8151783100662989
mse: 0.8151783113637948
mse: 0.815178312231703
mse: 0.8151783128112303
mse: 0.8151783131977727
mse: 0.815178313455393
mse: 0.8151783136269443
mse: 0.8151783137411694
mse: 0.8151783138171883
mse: 0.8151783138677731
mse: 0.8151783139014225
mse: 0.8151783139238268
mse: 0.8151783139387164
mse: 0.8151783139486178
mse: 0.8151783139552049
mse: 0.815178313959584
mse: 0.8151783139625046
mse: 0.8151783139644463
mse: 0.8151783139657234
mse: 0.8151783139665968
mse: 0.8151783139671547
mse: 0.8151783139675339
mse: 0.8151783139677934
mse: 0.815178313967960

In [106]:
predictions_rating = open("predictions_rating_Ass_1_11.18.txt", 'w')
for l in open("pairs_rating.txt"):
   if l.startswith("reviewerID"):
   #header
      predictions_rating.write(l)
      continue
   u,i = l.strip().split('-')
   rting = alpha + beta_u[u] +beta_i[i]+gamma_u[u]*gamma_i[i]
   print(rting)
   if rting>5 :
        rting = 5
   predictions_rating.write(u + '-' + i + ',' + str(rting) + '\n')
predictions_rating.close()

4.0929694077680265
3.973009097065918
4.089750633835035
4.250289502615711
4.232384909372539
4.843754934336228
4.485199853520149
4.485481482535129
4.661418705357799
3.474928799818261
4.4254832832914754
3.806238567488698
4.657859230090764
4.013187347837774
4.190753529757659
4.396116734110488
4.66235688550824
4.281235568358289
4.0255913387471125
2.588730869672894
3.668192978518221
3.8671458513109074
3.448094406944322
3.851521698170191
4.305328130585158
4.607037256266933
4.607904116294605
4.049765730727808
3.7750327943613984
4.429750418992007
4.378841785114726
4.240403179662053
4.10656207957329
3.502603699363334
4.780121436507996
3.9260327757146043
4.030795656168183
4.20718531417037
4.704486707489211
3.923446380455569
4.557530869132834
4.487922948502184
3.912159951014753
4.328242427284457
4.204238430193022
4.550816278773288
4.023803567345968
4.668680669025552
3.66814989668076
4.0771618291519705
4.296915758376851
4.623428388386916
4.105926020867461
4.462027644766976
4.320642156369986
4.33061

3.546189719237346
4.347050778791153
4.135598419510282
3.9888003146804296
4.262588264972034
4.12456131002534
4.432201260936349
4.114558431497519
4.417690316997784
3.575597114206835
4.8291183844828725
4.382770548644369
4.628314779374682
4.12346347284316
4.268962449236967
4.416478238585618
4.283562632482715
4.004274957446295
4.204829276117747
4.902977070859524
3.771462605074588
4.6458728556198325
4.763549185061548
3.954680374036188
4.205571884688697
3.72249487944793
4.702902223339171
4.949268066452712
4.492047775732639
4.462140151251431
4.425696835253521
4.31536844381569
4.498922995697889
3.8517283672312668
4.6323997114706374
4.5023205004702165
4.197037807003748
4.778295429737276
4.507925870793216
4.087644167071486
4.33421570565553
4.611180423486882
3.9724129268674235
3.819420629376253
4.03395355771894
3.146144555356775
4.594918484690924
4.145519633068416
4.386336051518855
4.163188685080111
4.507760067586425
4.164279865960397
3.749285009441317
4.209425984847248
4.4566798238730465
4.624347

3.9461170606762943
3.9072266734274246
4.482940187732676
4.4478220576732
4.139305775414428
4.813864015706165
4.386662598448537
4.023257082458388
4.049054702136631
4.097318419635742
4.709823195355312
4.213848527963066
4.089582674327771
4.797237063617256
4.247929951243272
4.282210965306168
3.751403588019546
4.248184441332543
3.572603635232378
4.665453119214666
4.165636266550505
4.570141695411107
4.893469524009438
3.718226195216384
4.083261271002769
4.445968602888236
3.520577351097826
4.488952634751048
3.5358277095820134
3.74130445423392
4.075444610561922
4.3311749412761404
3.9864714289053427
4.196759386868067
3.8014008038001874
4.535736082482832
4.7084722309599565
4.6322199568877265
4.003981769309676
3.955762627328185
4.393905197732291
4.034202441701661
4.329411502903015
4.387664018194223
4.5927203910229695
4.344803901281787
4.5952898846244725
3.795756396198766
3.695844199612248
4.339449754519805
4.039783363599439
4.859129456754445
4.394188777006799
3.6571219010530225
4.629371058879703
4.

4.142793628475677
2.8087373879792126
4.430996041249696
4.22644451032701
4.12238537460237
4.237264335505919
4.440170675238704
3.7799608235258457
3.1621263225094784
4.0071515240945725
4.02328202742674
4.03229168038454
4.14587528076405
4.684543422740495
4.4405880470787995
4.502106012865439
4.104095925757975
4.4147340494453
4.731502807658909
3.8326937812244974
4.395023853787969
4.609444879338134
4.671778704541648
4.435734394054811
3.730586713331837
4.528874532616806
3.629091109385645
4.871327087394702
4.762864625216083
4.769555849318392
4.551269079822481
3.7971476113788922
4.2643138987400375
4.406786679037061
4.236179911562902
4.474505777727651
4.137313256225567
4.204559671100522
4.735029482538093
3.591937906310882
4.684283214764433
3.962651720644994
3.853842789144566
3.802505775791456
4.4965476691605035
4.471835130781163
4.820522770599467
4.13578625735455
4.37236422069719
4.366960804394381
4.462579239025797
4.330604421431824
4.113923045199885
4.125043169239171
4.0711410292354175
3.6630535

4.405542895089529
4.173927857758554
4.512564157899164
4.370588246374879
4.615850374614468
4.543343704732413
3.878241524208397
3.7359825574126058
4.355905557588071
3.683867938046005
3.4480537975942314
4.598231643240145
4.58626923218141
4.3928887988370695
4.9049037172822905
4.674680723871644
4.216009823479157
3.967732829002855
4.326675497618
4.000254109374989
4.489844763455945
4.570034596395443
4.102227132516093
4.230067057936516
4.8814278269575935
4.502004149816366
2.7243990776148546
4.518533791979145
4.278345124377278
3.99418485444772
4.6255584702884756
4.504735387039754
4.2865737401704225
3.0814645125994335
4.412812293889811
4.787208550283576
4.435609474777734
4.3602501827685725
3.992401536674345
3.3003415722521905
4.2900629286443355
3.449987604200541
4.274956632358052
4.534771545179516
4.509067377796701
4.469465847863643
4.571411063495265
4.761287087527771
4.300377031502254
4.29300590660979
4.214426609321705
4.197840353985498
4.5427761004942075
4.075718112948574
4.175934167564941
4.2

4.332436903628487
4.530348902123338
3.3002849695186995
4.152569435130838
4.647321354776913
4.025471118691762
4.2106716617052635
3.9906070922466346
4.08704579010555
4.140696026454888
4.587435176433785
3.725731828315773
3.7991122484234516
3.5141935000747067
4.013596348736304
3.902313115834201
4.287517229677598
4.300303667364653
3.171845853474833
4.404714205662144
4.075174249920501
4.494138469735115
4.517246111557189
4.667310668102449
4.655383176917833
4.185340912096403
4.452869681224373
4.53334672325681
4.5783301081836045
4.34724576853847
4.415478334632138
4.212008416825229
3.925668060746801
4.030421227992724
4.032208141017545
4.066241184461963
4.055071936900605
4.460294740795152
3.744732152919681
4.447399876869821
4.2687614054788865
4.186107746911645
3.76758875531802
3.648302273669663
4.818415894717295
4.625800256700561
4.830367045535044
4.5188207612934885
3.9083478367419673
4.439773997887653
3.9720213577682792
4.693509302376622
4.4469169207900086
4.332889520672676
4.07001174497103
3.84

3.7399870462039284
4.428579941496143
4.527738788411424
4.943927432236999
4.6883444587590475
4.618065607714916
4.3770388115498
3.6026752632816392
3.2118451325590005
4.0927254054339635
4.436221656914058
4.579810718667257
4.274818340992796
4.658207946508018
4.418093236112115
4.299917617079869
4.132836645688204
4.375386555487525
4.486809305138532
4.497576359408601
4.219166778761935
4.526977541165449
3.6576774485042667
4.4356727563891845
4.19777753232557
3.955011758063396
4.947044981452573
4.103397042187724
4.436875559077705
4.911207179268247
4.329136042207441
4.04482243648834
3.8120263650304893
4.349578572872296
4.615556816562702
3.966305448223855
4.817560383693801
4.225870013555817
4.458898984743669
3.9328297361134017
4.366656684287015
4.137299085683007
4.482260034310611
4.845635214888919
4.3841698000138365
4.0503220759516525
3.738337780659223
4.732042224041873
4.175533416298668
4.248725345589246
4.257501501326685
4.717305291136342
4.507683785178257
4.409850735758533
4.3770656731671655
3.